In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ssl
import re
import os
import base64
import secrets
from flask import Flask, request, jsonify, render_template
from flask_wtf import FlaskForm, CSRFProtect
from wtforms import FileField, SubmitField
from PIL import Image
import easyocr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize Flask app
app = Flask(__name__, template_folder='/Users/jayaasri/Desktop/pro/templates')
csrf = CSRFProtect(app)

# Generate a secure secret key
app.config['SECRET_KEY'] = secrets.token_hex(16)

# Create a Flask-WTF form
class ImageUploadForm(FlaskForm):
    image = FileField('Image')
    submit = SubmitField('Analyze Image')

UPLOAD_FOLDER = 'static/images'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Disable SSL certificate verification for the EasyOCR model download
def disable_ssl_verification():
    if hasattr(ssl, '_create_unverified_context'):
        ssl._create_default_https_context = ssl._create_unverified_context

disable_ssl_verification()

# Load your toxicology dataset (ensure it has 'Ingredients' and 'Toxic' columns)
toxicology_df = pd.read_excel('ToxicologyDataset.xlsx')

# Preprocess the data
X = toxicology_df['Ingredients'].values
y = toxicology_df['Toxic'].values

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Pad sequences to a fixed length
X_padded = pad_sequences(X_seq)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build a simple LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X_padded.shape[1]))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Model Accuracy: {accuracy}")

# Define your routes and functions
@app.route('/home.html')
def cosmetic_analyze():
    return render_template('home.html')

@app.route('/login.html')
def login():
    return render_template('login.html')

@app.route('/signup.html')
def signup():
    return render_template('signup.html')

@app.route('/')
def index():
    form = ImageUploadForm()
    return render_template('CosmeticAnalyze.html', form=form, results=None)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'image' not in request.files:
        return "No file part"

    file = request.files['image']

    if file.filename == '':
        return "No selected file"

    if file and allowed_file(file.filename):
        filename = "test2.jpeg"  # Set the desired filename
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return "File uploaded successfully"

    else:
        return "Invalid file type. Allowed extensions are png, jpg, jpeg, gif."

@app.route('/get_image_and_text', methods=['GET'])
def get_image_and_text():
    IMAGE_PATH = 'static/images/test2.jpeg'

    with open(IMAGE_PATH, 'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode('utf-8')

    reader = easyocr.Reader(['en'])
    results = reader.readtext(IMAGE_PATH, paragraph=False)

    # Convert results to a list of strings
    results_str = [result[1] for result in results]

    # Tokenize and pad the sequences
    results_seq = tokenizer.texts_to_sequences(results_str)
    results_padded = pad_sequences(results_seq, maxlen=X_padded.shape[1])

    # Predict toxicity using the trained model
    predictions = model.predict(results_padded)
    toxic_predictions = (predictions > 0.5).astype(int)

    # Decode the toxic predictions
    toxic_labels = label_encoder.inverse_transform(toxic_predictions.flatten())

    return jsonify(results=results_str, toxic_predictions=list(toxic_labels))

if __name__ == '__main__':
    app.run(debug=True)



ModuleNotFoundError: ignored